# Tutorial: Wulff shapes (advanced level)

In [ ]:
%%capture

import os, re
import siman #program package to manage DFT calculations https://github.com/dimonaks/siman
from siman.calc_manage import smart_structure_read, get_structure_from_matproj
from siman.calc_manage import add, res
# Update configurations
from siman import header
from siman.database import write_database, read_database
from siman.set_functions import read_vasp_sets
from siman.header import db
from siman.header import _update_configuration
_update_configuration('../project_conf.py')
read_database() # read saved database if available
from pydoc import importfile
project_sets = importfile('../project_sets.py')
varset = read_vasp_sets(project_sets.user_vasp_sets, override_global = 1) #read user sets

from siman import thermo

header.PATH2PROJECT = 'icys_2024/tutorial_3_wulff_shapes'
header.PATH2EDITOR = 'notepad.exe'

from matplotlib import rc

import matplotlib.pyplot as plt
%matplotlib inline
# plt.rcParams['figure.figsize'] = [3.0, 2.2]
plt.rcParams['figure.dpi'] = 300


In [ ]:
from matplotlib import rc

import matplotlib.pyplot as plt
%matplotlib inline
# plt.rcParams['figure.figsize'] = [3.0, 2.2]
plt.rcParams['figure.dpi'] = 300

from matplotlib.transforms import blended_transform_factory



In [ ]:
# Only for this tutorial

import csv
from siman.geo import create_surface2, replic
import numpy as np
import matplotlib.patches as patches
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D

from IPython.core.display import Image, display
from tqdm import tqdm

# XRD patterns

In [ ]:
from pymatgen.analysis.diffraction.xrd import XRDCalculator, ATOMIC_SCATTERING_PARAMS


In [ ]:
vo2 = smart_structure_read("data/vo2.POSCAR")

vo2_pymatgen = vo2.convert2pymatgen()


In [ ]:
# XRD calculator and its settings
XRDCalculator.AVAILABLE_RADIATION
xrdc = XRDCalculator(wavelength="CuKa", symprec=1e-10)


In [ ]:
xrd_pattern = xrdc.get_pattern(vo2_pymatgen)

int_list = xrd_pattern.as_dict()["y"]
theta_list = xrd_pattern.as_dict()["x"]

hkl_list = []
for mill_idx in xrd_pattern.as_dict()["hkls"]: 
    hkl_cur = mill_idx[0]['hkl'] 
    hkl_cur = [str(hkl) for hkl in hkl_cur ]
    hkl_cur = "".join(hkl_cur)
    hkl_list.append(hkl_cur)

    

In [ ]:
xrdc.show_plot(vo2_pymatgen)

### Calculated vs [Experimental](https://www.sciencedirect.com/science/article/abs/pii/S004060902030448X) XRD patterns



In [ ]:
import pandas as pd


In [ ]:
exp_data = pd.read_csv('data/xrd_vo2.csv', names=["theta", "intensity"])
exp_data = exp_data.iloc[2:]

In [ ]:
exp_data.head()


In [ ]:
exp_intensity = [ float(ints) for ints in exp_data["intensity"].to_list() ]
exp_theta = [ float(thet) for thet in exp_data["theta"].to_list() ]

int_scale = max(theta_list) / max(exp_theta)

In [ ]:
# Data visualization

fig, ax = plt.subplots(figsize=(6, 2.5))

fontsize = 12
ax.set_xlabel(r"$2\Theta$, $^{\circ}$", fontsize=fontsize)
ax.set_ylabel("Intensity, arb. units", fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)
ax.set_xlim([20, 60])

# experimental values
ax.plot(np.array(exp_theta), np.array(exp_intensity)*int_scale, '-', c="black", label=r"Xiaoju, 2020 ($T=300$ K)")

# calculated values
ax.bar(np.array(theta_list), np.array(int_list), color="red", width=0.4, label=r"Calculated ($T=0$ K)")

# Annotate at the relative positions
tform = blended_transform_factory(ax.transData, ax.transAxes)
y_shift = 2.5

for idx, theta in enumerate(theta_list):
    ax.annotate(hkl_list[idx], (theta, int_list[idx]+y_shift), fontsize=fontsize-4,
            ha='center', va='center', color='blue', )

ax.legend(fontsize=fontsize-4, edgecolor="black")

plt.show()


## Surface energies with universal potentials

In [ ]:
def parse_chemical_formula(formula):
    # Regular expression to match element symbols followed by an optional number
    pattern = r"([A-Z][a-z]*)(\d*)"
    elements = re.findall(pattern, formula)
    
    # Convert the matches to a dictionary with element counts
    parsed_formula = {}
    for (element, count) in elements:
        parsed_formula[element] = int(count) if count else 1  # Default to 1 if count is missing

    return parsed_formula


In [ ]:
# !pip install ase -U

In [ ]:
# install MLUP(machine-learning universal potentials). Here, we use SevenNet (https://github.com/MDIL-SNU/SevenNet)

from sevenn.sevennet_calculator import SevenNetCalculator

In [ ]:
from ase.optimize import FIRE  # read more here: https://wiki.fysik.dtu.dk/ase/_modules/ase/optimize/fire.html
from ase.io import read, write


In [ ]:
sevennet_0_cal = SevenNetCalculator("7net-0", device='cpu') # we use CPUs for our calculations. 
                                                            # GPUs with CUDA are also an option.


In [ ]:
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.vis.structure_vtk import StructureVis

from pymatgen.io.ase import AseAtomsAdaptor
from ase.visualize import view
from ase.visualize.plot import plot_atoms

from pymatgen.core.surface import get_symmetrically_distinct_miller_indices
from pymatgen.core.surface import SlabGenerator


In [ ]:
zr3o = db['zr3o.su', "bulk_eos", 100].copy().end
zr3o_pmg = zr3o.convert2pymatgen()

In [ ]:
mil_list = get_symmetrically_distinct_miller_indices(structure = zr3o_pmg, max_index = 2, return_hkil = False)

In [ ]:
en_list = []
name_list = []
formula_list = []
suf_area_list = []

for mil_idx in tqdm(mil_list):
    slabgen = SlabGenerator(initial_structure = zr3o_pmg, miller_index = mil_idx, min_slab_size = 10, 
                            min_vacuum_size = 10, lll_reduce = True, center_slab = False, primitive = False)

    # Number of terminations for the given Miller index
    slabs = slabgen.get_slabs()

    # Current miller index as string 
    mil_cur = [ str(x) for x in mil_idx ]  
    mil_cur = "".join(mil_cur)

    for idx, slab in enumerate(slabs):
        calc_name = mil_cur + "." + str(idx)
        st_read = read("data/zr3o_surfaces/" + calc_name +".POSCAR" )

        if 0:
            # uncomment if you want to perform calculations by your own
            st_read.calc = sevennet_0_cal
            optim = FIRE(st_read)
            optim.run(fmax = 0.01)
    
            name_list.append(calc_name)
            en_list.append(optim.optimizable.get_potential_energy())
            formula_list.append(st_read.get_chemical_formula())
            suf_area_list.append(np.linalg.norm(np.cross(st_read.get_cell()[0], st_read.get_cell()[1])))

    

## Calculate Zr3O and O2 energies with SevenNet 

In [ ]:
zr3o = db["zr3o.su", "bulk_eos", 100].copy().end

o2 = db["o2", "o2_box", 1].copy().end


In [ ]:
zr3o = zr3o.convert2pymatgen()
zr3o = AseAtomsAdaptor.get_atoms(zr3o)

o2 = o2.convert2pymatgen()
o2 = AseAtomsAdaptor.get_atoms(o2)


In [ ]:
# Calculate energies of O2 and ZrO phases
if 1:
    zr3o.calc = sevennet_0_cal
    optim_zr3o = FIRE(zr3o)
    optim_zr3o.run(fmax = 0.01)
    
    o2.calc = sevennet_0_cal
    optim_o2 = FIRE(o2)
    optim_o2.run(fmax = 0.01)



In [ ]:
# Append data on Zr3O and O2 to csv file
if 0:
    with open('data/zr3o_sevennet_all.csv', 'a') as f:
        o2_data = "o2" + "," + str(optim_o2.optimizable.get_potential_energy()) + "," + o2.get_chemical_formula() + "\n" 
        zr3o_data = "zr3o" + "," + str(optim_zr3o.optimizable.get_potential_energy()) + "," + zr3o.get_chemical_formula() + "\n"  
        
        f.write(o2_data)
        f.write(zr3o_data)


## Visualization of data, which were calculated for you and located in data/

In [ ]:
import pandas as pd

In [ ]:
data_snn = pd.read_csv('data/zr3o_sevennet_all.csv', names=["surface", "energy", "formula", "area"])

data_dft = pd.read_csv('data/zr3o_dft.csv', names=["surface", "energy", "formula", "area"])


In [ ]:
# SNN data
row_o2_snn = data_snn.loc[data_snn["surface"] == "o2"]
data_o2_snn = row_o2_snn.values[0]

row_zr3o_snn = data_snn.loc[data_snn["surface"] == "zr3o"]
data_zr3o_snn = row_zr3o_snn.values[0]

# DFT data
row_o2_dft = data_dft.loc[data_dft["surface"] == "o2"]
data_o2_dft = row_o2_dft.values[0]

row_zr3o_dft = data_dft.loc[data_dft["surface"] == "zr3o"]
data_zr3o_dft = row_zr3o_dft.values[0]



In [ ]:
# Drop lines with O2 and Zr3O
data_snn = data_snn[data_snn["surface"] != "o2" ]
data_snn = data_snn[data_snn["surface"] != "zr3o" ]

data_dft = data_dft[data_dft["surface"] != "o2" ]
data_dft = data_dft[data_dft["surface"] != "zr3o" ]


# Plot surface energies

In [ ]:
T_points = 6

T_min = 200
T_max = 1000
T_list = np.linspace(T_min, T_max, T_points) 
P = 0.21 # partial pressure of oxygen during synthesis

o_list_dft = []
zr_list_dft = []
e_zr3o_dft = float(data_zr3o_dft[1]) / 2
e_o_dft = float(data_o2_dft[1]) / 2

o_list_snn = []
zr_list_snn = []
e_zr3o_snn = float(data_zr3o_snn[1]) / 2
e_o_snn = float(data_o2_snn[1]) / 2

for T in T_list:
    o_chem_dft =  e_o_dft + (thermo.O2(T=T, c2ev=1, P=P)[0] / 2) + 0.68
    zr_chem_dft = (e_zr3o_dft - o_chem_dft) / 3
    o_list_dft.append(o_chem_dft)
    zr_list_dft.append(zr_chem_dft)

    # For SevenNet
    o_chem_snn = e_o_snn + (thermo.O2(T=T, c2ev=1, P=P)[0] / 2) + 0.68
    zr_chem_snn = (e_zr3o_snn - o_chem_snn) / 3
    o_list_snn.append(o_chem_snn)
    zr_list_snn.append(zr_chem_snn)

    
# Plot data
fig, ax = plt.subplots(figsize=(3, 2.2) )
fontsize = 7
ax.set_xlabel("Temperature, K", fontsize=fontsize)
ax.set_ylabel("Chemical potential, eV", fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)

# DFT results
ax.plot(T_list, o_list_dft, '--', c='r', label=r"$\mu_{\mathrm{O}}$ (DFT)")
ax.plot(T_list, zr_list_dft, '--', c='b', label=r"$\mu_{\mathrm{Zr}}$ (DFT)")

# SNN results
ax.plot(T_list, o_list_snn, '-', c='r', label=r"$\mu_{\mathrm{O}}$ (SNN)")
ax.plot(T_list, zr_list_snn, '-', c='b', label=r"$\mu_{\mathrm{Zr}}$ (SNN)")

# Synthesis temperatures are 240-350 C, see https://doi.org/10.1016/j.jssc.2013.10.023
ylim = ax.get_ylim()
xlim = ax.get_xlim()
ax.set_ylim(ylim[0], ylim[1]*1.05)
ylim = ax.get_ylim()

p = patches.Rectangle((240+273, ylim[0]), (350-240), ylim[1]-ylim[0], linewidth=0, alpha=0.1, facecolor='none', zorder=1)
p.set_color('black')
ax.add_patch(p)
ax.set_xlim([T_min, T_max])

ax.legend(fontsize=fontsize-1, edgecolor="black")
fig.tight_layout()
fig.show()


In [ ]:
# Synthesis conditions
T_synth_snn = T_list[2] 
T_synth_dft = T_list[2] 

chem_list_snn = {"O": o_list_snn[2], "Zr": zr_list_snn[2]}
chem_list_dft = {"O": o_list_dft[2], "Zr": zr_list_dft[2]}

print("Let's say that we want synthesis at temperature of {} K".format(T_synth_snn))


In [ ]:
# For SevenNet
names_list_snn = []
tot_en_list_snn = []
suf_en_list_snn = []
fml_list_snn = []
suf_area_list_snn = []


for index, row in data_snn.iterrows():
    suf_en_cur = float(row.values[1])          
    fml_cur = parse_chemical_formula( row.values[2] ) 
    suf_area_cur = float(row.values[3])          

    for el in ["Zr", "O"]:
        suf_en_cur -= fml_cur[el] * chem_list_snn[el]

    suf_en_cur /= 2*suf_area_cur
    suf_en_cur *= header.eV_A_to_J_m

    suf_en_list_snn.append(suf_en_cur)
    tot_en_list_snn.append(float(row.values[1]))
    names_list_snn.append(row.values[0])
    fml_list_snn.append(fml_cur)
    suf_area_list_snn.append(suf_area_cur)

# For DFT
names_list_dft = []
tot_en_list_dft = []
suf_en_list_dft = []
fml_list_dft = []
suf_area_list_dft = []

for index, row in data_dft.iterrows():
    suf_en_cur = float(row.values[1])          
    fml_cur = parse_chemical_formula( row.values[2] ) 
    suf_area_cur = float(row.values[3])          

    for el in ["Zr", "O"]:
        suf_en_cur -= fml_cur[el] * chem_list_dft[el]

    suf_en_cur /= 2*suf_area_cur
    suf_en_cur *= header.eV_A_to_J_m

    suf_en_list_dft.append(suf_en_cur)
    tot_en_list_dft.append(float(row.values[1]))
    names_list_dft.append(row.values[0])
    fml_list_dft.append(fml_cur)
    suf_area_list_dft.append(suf_area_cur)
    


In [ ]:
# Sufrace and total energies for SevenNet 

dir_suf_snn = {}
dir_en_snn = {}

for idx, name in enumerate(names_list_snn):
    name_cur = name.split(".")[0]
    if name_cur not in dir_suf_snn.keys():
        dir_suf_snn[name_cur] = [suf_en_list_snn[idx]]
        dir_en_snn[name_cur] = [tot_en_list_snn[idx]]
    else:
        dir_suf_snn[name_cur].append(suf_en_list_snn[idx])
        dir_en_snn[name_cur].append(tot_en_list_snn[idx])
    
# Sufrace and total energies for DFT 
dir_suf_dft = {}
dir_en_dft = {}

for idx, name in enumerate(names_list_dft):
    name_cur = name.split(".")[0]
    if name_cur not in dir_suf_dft.keys():
        dir_suf_dft[name_cur] = [suf_en_list_dft[idx]]
        dir_en_dft[name_cur] = [tot_en_list_dft[idx]]
    else:
        dir_suf_dft[name_cur].append(suf_en_list_dft[idx])
        dir_en_dft[name_cur].append(tot_en_list_dft[idx])    


In [ ]:
fig, (ax1, ax2) = plt.subplots(figsize=(5, 4.5), ncols=1, nrows=2 )
fontsize = 7

ax1.set_title("DFT results", fontsize=fontsize+2)
ax1.set_ylabel(r"Surface energy, J/m$^2$", fontsize=fontsize+2)
ax1.xaxis.set_tick_params(labelsize=fontsize)
ax1.yaxis.set_tick_params(labelsize=fontsize)

ax2.set_title("SevenNet results", fontsize=fontsize+2)
ax2.set_ylabel(r"Surface energy, J/m$^2$", fontsize=fontsize+2)
ax2.xaxis.set_tick_params(labelsize=fontsize)
ax2.yaxis.set_tick_params(labelsize=fontsize)


labels, data_plot = dir_suf_dft.keys(), dir_suf_dft.values()

ax2.boxplot(data_plot)
ax2.set_xticks(range(1, len(labels) + 1), labels, fontsize=fontsize)
ax2.set_xlabel("Surface orientation", fontsize=fontsize+2)

labels, data_plot = dir_suf_snn.keys(), dir_suf_snn.values()

ax1.boxplot(data_plot)
ax1.set_xticks(range(1, len(labels) + 1), labels, fontsize=fontsize)
ax1.set_xlabel("Surface orientation", fontsize=fontsize+2)

fig.tight_layout()
plt.show()


# Compare DFT vs SevenNet results

## Slabs' total energies

In [ ]:
colors_list = ["tomato", "royalblue", "magenta", "lawngreen", "cyan", "silver", "purple",
               "pink", "peru", "black", "forestgreen", "white", "orange", "indigo"]

In [ ]:
diff_list = []

fig, ax = plt.subplots(figsize=(2, 3.2), nrows=2, ncols=1 )

fontsize = 6
for i in [0,1]:
    ax[i].set_xlabel("DFT total energy, eV", fontsize=fontsize)
    ax[i].set_ylabel("SevenNet total energy, eV", fontsize=fontsize)
    ax[i].xaxis.set_tick_params(labelsize=fontsize)
    ax[i].yaxis.set_tick_params(labelsize=fontsize)
    
    for idx, key in enumerate(dir_en_dft.keys()):
        values_dft = dir_en_dft[key]
        values_snn = dir_en_snn[key]
        ax[i].scatter(values_dft, values_snn, c=colors_list[idx], s=10, label=key, edgecolor="black", zorder=2)
        diff_list += (np.array(values_dft) - np.array(values_snn)).tolist()  # difference between two methods
    
    ylim = ax[i].get_ylim()
    xlim = ax[i].get_xlim()
    
    ax[i].plot([xlim[0], xlim[1]], [xlim[0], xlim[1]], '-', c="red", lw=1.0, zorder=1  )
    ax[i].legend(fontsize=fontsize-2, edgecolor="black", ncols=3)
    ax[i].plot([xlim[0], xlim[1]], [xlim[0], xlim[1]], '-', c="red", lw=1.0, zorder=1  )

    if (i == 1):
        lims_cut = [-280, -265]
        ax[i].set_ylim(lims_cut)
        ax[i].set_xlim(lims_cut)
        

plt.show()



In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 1.5) )

fontsize = 7
ax.set_xlabel("Number of ticks", fontsize=fontsize)
ax.set_ylabel(r"$\Delta$(DFT vs. SevenNet), eV", fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)

ax.hist(diff_list, bins=20, rwidth=5, color="royalblue", edgecolor="black")

plt.show()


## Surface energies

In [ ]:
diff_list = []
idx_list_snn = []
idx_list_dft = []

fig, ax = plt.subplots(figsize=(3, 1.6) )

fontsize = 6
ax.set_xlabel(r"DFT surface energies, J/m$^2$", fontsize=fontsize)
ax.set_ylabel(r"SeveNet surface energies, J/m$^2$", fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)

for idx, key in enumerate(dir_suf_dft.keys()):
    values_dft = dir_suf_dft[key]
    list_temp = list(range(len(values_dft)))
    list_temp = [x for _,x in sorted(zip(values_dft,list_temp))]
    idx_list_dft.append(list_temp)
    values_snn = dir_suf_snn[key]
    list_temp = list(range(len(values_snn)))
    list_temp = [x for _,x in sorted(zip(values_snn,list_temp))]
    idx_list_snn.append(list_temp)
    # plot data
    ax.scatter(values_dft, values_snn, c=colors_list[idx], s=20, label=key, edgecolor="black", zorder=2)
    diff_list += (np.array(values_dft) - np.array(values_snn)).tolist()  #difference between two methods

ylim = ax.get_ylim()
xlim = ax.get_xlim()

ax.plot([xlim[0], xlim[1]], [xlim[0], xlim[1]], '-', c="red", lw=1.0, zorder=1  )

ax.legend(fontsize=fontsize-2, edgecolor="black", ncols=3)
plt.show()



In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 1.5) )

fontsize = 7
ax.set_xlabel("Number of ticks", fontsize=fontsize)
ax.set_ylabel(r"$\Delta$(DFT vs. SevenNet), J/m$^2$", fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)

ax.hist(diff_list, bins=20, rwidth=5, color="royalblue", edgecolor="black")

plt.show()


# How well can we predict the most stable termination?

In [ ]:
num_steps = 4
conf_values = []

for step in list(range(1, num_steps+1, 1)):
    bool_ind = []
    for idx, miller in enumerate(idx_list_dft):
        idx_min = miller[0]
        # if min termination falls into the specified range
        if ( step <= len(idx_list_snn[idx]) ):
            if (idx_min in idx_list_snn[idx][0:step]) :
                bool_ind.append(1)
            else:
                bool_ind.append(0)
        else:
            bool_ind.append(1)
            
    conf_values.append(bool_ind)
    


In [ ]:
conf_percent = [sum(idx)/len(idx) for idx in conf_values]

In [ ]:
fig, ax = plt.subplots(figsize=(2.5, 1.5) )

fontsize = 6
ax.set_ylabel("Correctly found terminations, %", fontsize=fontsize)
ax.set_xlabel(r"Termination confidence interval", fontsize=fontsize)
ax.xaxis.set_tick_params(labelsize=fontsize)
ax.yaxis.set_tick_params(labelsize=fontsize)

ax.bar(list(range(1, num_steps+1, 1)), np.array(conf_percent)*100, width=0.5, edgecolor="black", color="royalblue", )

plt.show()


# Wulff shapes: DFT vs MLUP 

In [ ]:
from pymatgen.analysis.wulff import WulffShape



In [ ]:
def calc_suf_en(en, ch_fml, suf_area, chem_pots):
    """
    INPUTS:
        en (float) - total energy of a slab
        ch_fml (dir) - dictionary of chemical elements in structure 
        suf_area_list_dft (float) - surface energies
        chem_pots (dir) - dictionary of chemical potentials
    RETURNS:
        suf_en (float) - surface energy in J/m2
    """
    
    suf_en = en
    for el in chem_pots.keys():
        suf_en -= chem_pots[el] * ch_fml[el]   

    suf_en /= 2*suf_area
    suf_en *= header.eV_A_to_J_m

    return suf_en

    

In [ ]:
def parse_mil_dir_to_list(mil_dir):
    # Parse list of miller indices (list of str) to list (2D array)  
    mil_list = []
    
    for mil in mil_dir.keys():
        mils_cur = []
        mil_cur = ""
        for char in mil:
            if (char == "-"):
                mil_cur += char
            else:
                mil_cur += char
                mils_cur.append(int(mil_cur))
                mil_cur = ""
        mil_list.append(mils_cur)

    return mil_list

In [ ]:
st = db["zr3o.su", "bulk_eos", 100].copy().end
zr3o_pmg = st.convert2pymatgen()

In [ ]:
"""
    Just to remind to you
    names_list_dft - list with names
    tot_en_list_dft - list with total energies
    fml_list_dft - list with chemical formulas
    suf_area_list_dft - list with surface energies
"""

# Calculation of chemical potentials
e_zr3o_snn = float(data_zr3o_snn[1]) / 2
e_o_snn = float(data_o2_snn[1]) / 2

e_zr3o_dft = float(data_zr3o_dft[1]) / 2
e_o_dft = float(data_o2_dft[1]) / 2

T_list = [300]

for T_cur, T in enumerate(tqdm(T_list)):
    # For DFT
    chem_list_dft = {}
    o_chem_dft =  e_o_dft + (thermo.O2(T=T, c2ev=1, P=P)[0] / 2) + 0.68
    zr_chem_dft = (e_zr3o_dft - o_chem_dft) / 3
    chem_list_dft["O"] = o_chem_dft
    chem_list_dft["Zr"] = zr_chem_dft
    
    # For SevenNet
    o_chem_snn = e_o_snn + (thermo.O2(T=T, c2ev=1, P=P)[0] / 2) + 0.68
    zr_chem_snn = (e_zr3o_snn - o_chem_snn) / 3
    chem_list_snn["O"] = o_chem_dft
    chem_list_snn["Zr"] = zr_chem_dft

    # CALCULATE ALL SURFACE ENERGIES FOR THE GIVEN CHEMICAL POTENTILAS
    wullf_suf_en_dft = {}
    wullf_suf_en_snn = {}

    for idx, name in enumerate(names_list_dft):
        # For DFT
        suf_en_dft = calc_suf_en( tot_en_list_dft[idx], fml_list_dft[idx], 
                                 suf_area_list_dft[idx], chem_list_dft ) 
        suf_orient = name.split(".")[0]   
        if (suf_orient not in wullf_suf_en_dft):
            wullf_suf_en_dft[suf_orient] = [suf_en_dft]
        else:
            wullf_suf_en_dft[suf_orient].append(suf_en_dft)

        # For SevenNet
        suf_en_snn = calc_suf_en( tot_en_list_snn[idx], fml_list_snn[idx],
                                 suf_area_list_snn[idx], chem_list_snn ) 
        suf_orient = name.split(".")[0]   
        if (suf_orient not in wullf_suf_en_snn):
            wullf_suf_en_snn[suf_orient] = [suf_en_snn]
        else:
            wullf_suf_en_snn[suf_orient].append(suf_en_snn)
        

    # MINIMAL SURFACE ENERGY FOR EACH MILLER INDEX
    suf_en_min_snn = {}
    suf_en_min_dft = {}

    for key in wullf_suf_en_snn.keys():
        suf_en_min_snn[key] = min(wullf_suf_en_snn[key])
        suf_en_min_dft[key] = min(wullf_suf_en_dft[key])
        
    # VISUALIZE WULLF SHAPES
    if 1:
        ax = WulffShape(zr3o_pmg.lattice, parse_mil_dir_to_list(suf_en_min_dft),
                             list(suf_en_min_dft.values()) ).get_plot(show_area=False, 
                    color_set="CMRmap_r", aspect_ratio=(8, 16), direction=(1,1,0.5))
    else:
        ax = WulffShape(zr3o_pmg.lattice, parse_mil_dir_to_list(suf_en_min_snn),
                             list(suf_en_min_dft.values()) ).get_plot(show_area=False, 
                    color_set="CMRmap_r", aspect_ratio=(8, 16), direction=(1,1,0.5))

        
    

# DFT vs SevenNet results

<div>
<center>
<img src="figures/wulff_shapes/dft_vs_snn.png" width="1000" height="400"/>
<center>
</div>
    
<center>
<b>Figure.</b> Calculated Wulff shapes (left) DFT; (right) SevenNet. 
<center>